The script is very similar to what Tomáš Brůna wrote. The difference is in the classifiers - DecisionTree and SGD Classifier.

In [2]:
path = ('/mnt/c/Users/Vasanta Chivukula/Documents/Vannbergclass/Challenge/train_pro.tsv')


cat: path: No such file or directory


In [14]:
import os
import sys
import getopt
import re
import csv
import pandas as pd
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier


In [15]:
import csv
labels = pd.read_csv('/mnt/c/Users/Vasanta Chivukula/Documents/Vannbergclass/Challenge/newchallenge/train_cli.tsv', sep="\t", index_col="sample")
proteins = pd.read_csv('/mnt/c/Users/Vasanta Chivukula/Documents/Vannbergclass/Challenge/newchallenge/train_pro.tsv', sep="\t")
# Transpose proteins matrix
proteins = proteins.T
misClassified = pd.read_csv("/mnt/c/Users/Vasanta Chivukula/Documents/Vannbergclass/Challenge/newchallenge/sum_tab_1.csv", sep=",")
# Replace missing values with median
proteins = proteins.fillna(proteins.median())
# Drop remaining columns with missing values
proteins = proteins.dropna(axis='columns')

proteins.to_csv('transpose_file.csv', index=False)

matches variable holds the index of only correctly labeled data from sum_tab_1. loc is a pandas function which helps locate the matches.Locating the protein sets for only those that have matches in the train files.

In [16]:
matches = list(misClassified.query('mismatch==0').loc[:,"sample"])
x = proteins.loc[matches]
y = labels.loc[matches]

Cross validating the classifier (splitting the train data into train and test to check if the classifier predicts correctly)

x_train - 70% data from pro file in train
X-test - 30% of the data from train used as test
y_train 70% is the data from cli
y_test is 30% oof data from cli

 y_gender_train = lb.fit_transform(y_train.loc[:,"gender"]).ravel() - fitting the linearBinarizer to y_train gender

In [17]:
def classify(x, y, clf):
    lb = preprocessing.LabelBinarizer()
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, shuffle=True, random_state=100)
    # I will have separat models for gender and msi
    y_gender_train = lb.fit_transform(y_train.loc[:,"gender"]).ravel()
    y_gender_test = lb.fit_transform(y_test.loc[:,"gender"]).ravel()
    y_msi_train = lb.fit_transform(y_train.loc[:,"msi"]).ravel()
    y_msi_test = lb.fit_transform(y_test.loc[:,"msi"]).ravel()

    clf.fit(x_train, y_gender_train)

    y_gender_predict = clf.predict(x_train)
    print("Gender train accuracy:", accuracy_score(y_gender_train, y_gender_predict))
    # print("Gender train F1:", f1_score(y_gender_train, y_gender_predict))

    y_gender_predict = clf.predict(x_test)
    print("Gender test accuracy:", accuracy_score(y_gender_test, y_gender_predict))
    # print("Gender test F1:", f1_score(y_gender_test, y_gender_predict))

    clf.fit(x_train, y_msi_train)

    y_msi_predict = clf.predict(x_train)
    print("Msi train accuracy:", accuracy_score(y_msi_train, y_msi_predict))
    # print("Msi train F1:", f1_score(y_msi_train, y_msi_predict))

    y_msi_predict = clf.predict(x_test)
    print("Msi test accuracy:", accuracy_score(y_msi_test, y_msi_predict))
    # print("Msi test F1:", f1_score(y_msi_test, y_msi_predict))

In [52]:
classify(x, y, SVC(C=100, kernel="rbf", gamma="scale", probability=True))

Gender train accuracy: 1.0
Gender test accuracy: 0.619047619048
Msi train accuracy: 1.0
Msi test accuracy: 1.0


In [53]:
classify(x, y, LogisticRegression(solver='newton-cg', max_iter=1000))

Gender train accuracy: 1.0
Gender test accuracy: 0.619047619048
Msi train accuracy: 1.0
Msi test accuracy: 1.0


In [54]:
classify(x, y, LogisticRegression(solver='lbfgs', max_iter=1000))

Gender train accuracy: 1.0
Gender test accuracy: 0.619047619048
Msi train accuracy: 1.0
Msi test accuracy: 1.0


In [55]:
classify(x, y, tree.DecisionTreeClassifier(random_state=1))

Gender train accuracy: 1.0
Gender test accuracy: 0.571428571429
Msi train accuracy: 1.0
Msi test accuracy: 0.714285714286


In [56]:
classify(x, y, KNeighborsClassifier())

Gender train accuracy: 0.872340425532
Gender test accuracy: 0.666666666667
Msi train accuracy: 0.893617021277
Msi test accuracy: 0.952380952381


In [18]:
test_labels = pd.read_csv('/mnt/c/Users/Vasanta Chivukula/Documents/Vannbergclass/Challenge/newchallenge/test_cli.tsv', sep="\t", index_col="sample")
test_proteins = pd.read_csv('/mnt/c/Users/Vasanta Chivukula/Documents/Vannbergclass/Challenge/newchallenge/test_pro.tsv', sep="\t")
test_proteins = test_proteins.T
test_proteins = test_proteins.fillna(proteins.median())
test_proteins = test_proteins.dropna(axis='columns')

In [25]:
def final_classify(x, y, test_prot, test_label, clf):
    lb = preprocessing.LabelBinarizer()
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, shuffle=True, random_state=100)
    # I will have separat models for gender and msi
    y_gender_train = y_train.loc[:,"gender"]
    y_gender_test = y_test.loc[:,"gender"]
    y_msi_train = y_train.loc[:,"msi"]
    y_msi_test = y_test.loc[:,"msi"]

    clf.fit(x_train, y_gender_train)

    y_gender_predict = clf.predict(x_train)
    print("Gender train accuracy:", accuracy_score(y_gender_train, y_gender_predict))
    # print("Gender train F1:", f1_score(y_gender_train, y_gender_predict))

    y_gender_predict = clf.predict(x_test)
    print("Gender test accuracy:", accuracy_score(y_gender_test, y_gender_predict))
    # print("Gender test F1:", f1_score(y_gender_test, y_gender_predict))


    clf.fit(x_train, y_msi_train)

    y_msi_predict = clf.predict(x_train)
    print("Msi train accuracy:", accuracy_score(y_msi_train, y_msi_predict))
    # print("Msi train F1:", f1_score(y_msi_train, y_msi_predict))

    y_msi_predict = clf.predict(x_test)
    print("Msi test accuracy:", accuracy_score(y_msi_test, y_msi_predict))
    # print("Msi test F1:", f1_score(y_msi_test, y_msi_predict))
    
    #lb = preprocessing.LabelBinarizer()

    y_gender = test_label.loc[:,"gender"]
    y_msi = test_label.loc[:,"msi"]

    clf.fit(x_train, y_gender_train)
    y_gender_predict = clf.predict(test_prot)

    print("Gender test accuracy:", accuracy_score( y_gender, y_gender_predict))
    probs = clf.predict_proba(test_prot)
    #for i in range(len(probs)):
        #print(probs[i] , y_gender_predict[i], y_gender[i])
    #print()
    clf.fit(x_train, y_msi_train)
    y_msi_predict = clf.predict(test_prot)
    print("Msi test accuracy:", accuracy_score(y_msi, y_msi_predict))
    probs = clf.predict_proba(test_prot)
    for i in range(len(probs)):
        row = (probs[i], y_gender_predict[i], y_gender[i], y_msi_predict[i], y_msi[i])
        
        with open('SGD_final_file.csv','a') as out:
            writer=csv.writer(out)
            writer.writerow(row)

    


In [20]:
final_classify(x,y, test_proteins, test_labels, KNeighborsClassifier())

Gender train accuracy: 0.872340425532
Gender test accuracy: 0.666666666667
Msi train accuracy: 0.893617021277
Msi test accuracy: 0.952380952381
Gender test accuracy: 0.4375
Msi test accuracy: 0.875


In [28]:
final_classify(x, y, test_proteins, test_labels, LogisticRegression(solver='newton-cg', max_iter=1000))

Gender train accuracy: 1.0
Gender test accuracy: 0.619047619048
Msi train accuracy: 1.0
Msi test accuracy: 1.0
Gender test accuracy: 0.3875
[ 0.88114746  0.11885254] Female Female
[ 0.94603902  0.05396098] Female Male
[ 0.97853094  0.02146906] Female Female
[ 0.97858197  0.02141803] Female Male
[ 0.06550025  0.93449975] Male Female
[ 0.82886665  0.17113335] Female Male
[ 0.37737788  0.62262212] Male Male
[ 0.38782122  0.61217878] Male Male
[ 0.98711809  0.01288191] Female Female
[ 0.5548236  0.4451764] Female Female
[ 0.98674807  0.01325193] Female Male
[ 0.99594412  0.00405588] Female Male
[ 0.16918631  0.83081369] Male Male
[ 0.91007303  0.08992697] Female Male
[ 0.66583386  0.33416614] Female Female
[ 0.98991702  0.01008298] Female Female
[ 0.99276121  0.00723879] Female Female
[ 0.92405734  0.07594266] Female Male
[ 0.22322319  0.77677681] Male Female
[  9.99862425e-01   1.37574689e-04] Female Male
[ 0.95607609  0.04392391] Female Female
[ 0.01065247  0.98934753] Male Female
[ 0.30

In [24]:
final_classify(x, y, test_proteins, test_labels, tree.DecisionTreeClassifier(random_state=1))

Gender train accuracy: 1.0
Gender test accuracy: 0.571428571429
Msi train accuracy: 1.0
Msi test accuracy: 0.714285714286
Gender test accuracy: 0.525
Msi test accuracy: 0.7375


In [26]:
final_classify(x, y, test_proteins, test_labels, SGDClassifier(loss='modified_huber', penalty='l1', alpha=0.0001, l1_ratio=0.15, fit_intercept=False, tol=0.001, epsilon=0.1, learning_rate='optimal', eta0=0.0, power_t=0.75, early_stopping=True, validation_fraction=0.1,max_iter=500, class_weight=None, warm_start=True, average=False, n_iter=None, n_iter_no_change=100))


Gender train accuracy: 0.957446808511
Gender test accuracy: 0.619047619048
Msi train accuracy: 0.978723404255
Msi test accuracy: 0.904761904762
Gender test accuracy: 0.375
Msi test accuracy: 0.8375
